# 유튜브 스크립트 불러오기

pip install langchain

In [1]:
from langchain.document_loaders import YoutubeLoader

pip install youtube_transcript_api

In [3]:
loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=Pn-W41hC764")
transcript = loader.load()
transcript

[Document(page_content="I alluded in my opening remarks to the the jobs issue the economic effects on employment uh I think you have said uh in fact and I'm going to quote development of superhuman machine intelligence is probably the greatest threat to the continued existence of humanity end quote you may have had in mind the effect on on jobs which is really my biggest nightmare in the long term uh let me ask you uh what your biggest nightmare is and whether you share that concern like with all technological revolutions I expect there to be significant impact on jobs but exactly what that impact looks like is very difficult to predict if we went back to the the other side of a previous technological Revolution talking about the jobs that exist on the other side um you know you can go back and read books of this it's what people said at the time it's difficult I believe that there will be far greater jobs on the other side of this and the jobs of today will get better I think it's imp

# 긴 내용 요약하기

In [5]:
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI

## 텍스트 쪼개기

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=4000, chunk_overlap=0)
text = text_splitter.split_documents(transcript)

In [7]:
len(text)

2

In [8]:
text[0]

Document(page_content="I alluded in my opening remarks to the the jobs issue the economic effects on employment uh I think you have said uh in fact and I'm going to quote development of superhuman machine intelligence is probably the greatest threat to the continued existence of humanity end quote you may have had in mind the effect on on jobs which is really my biggest nightmare in the long term uh let me ask you uh what your biggest nightmare is and whether you share that concern like with all technological revolutions I expect there to be significant impact on jobs but exactly what that impact looks like is very difficult to predict if we went back to the the other side of a previous technological Revolution talking about the jobs that exist on the other side um you know you can go back and read books of this it's what people said at the time it's difficult I believe that there will be far greater jobs on the other side of this and the jobs of today will get better I think it's impo

In [9]:
text[1]

Document(page_content="the less you can get a proper read on that so it's important first of all that scientists be part of that process and second that we have much greater transparency about what actually goes into these systems if we don't know what's in them then we don't know exactly how well they're doing when we give something new and we don't know how good a benchmark that will be for something that's entirely novel so I could go into that more but I want to flag that second is on jobs past performance history is not a guarantee of the future it has always been the case in the past that we have had more jobs that new jobs new professions come in as new technologies come in I think this one's going to be different and the real question is over what time time scale is it going to be 10 years is it going to be 100 years and I don't think anybody knows the answer to that question I think in the long run so-called artificial general intelligence really will replace a large fraction 

## 사용할 LLM 모델 설정

pip install openai
pip install tiktoken

In [12]:
llm = ChatOpenAI(temperature=0,
                    openai_api_key="API_key",
                    max_tokens=3000,
                    model_name="gpt-3.5-turbo",
                    request_timeout=120
                )

## 요약에 사용할 프롬프트

In [13]:
# 각각의 chunck 를 요약하기
prompt = PromptTemplate(
    template="""Summarize the youtube video whose transcript is provided within backticks \
    ```{text}```
    """, input_variables=["text"]
)

# 요약된 내용들을 취합하여 다시한번 요약하기
combine_prompt = PromptTemplate(
    template="""Combine all the youtube video transcripts  provided within backticks \
    ```{text}```
    Provide a concise summary between 8 to 10 sentences.
    """, input_variables=["text"]
)

## 요약 시작하기

In [14]:
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=False, map_prompt=prompt, combine_prompt=combine_prompt)

In [16]:
output = chain.run(text)

In [17]:
output

"The video explores the potential impact of superhuman machine intelligence on jobs and the economy. The speaker believes that while there will be significant changes in employment due to technological advancements, it is challenging to predict the exact consequences. They argue that previous technological revolutions have resulted in the creation of new and improved jobs, and they expect the same to occur with the development of superhuman machine intelligence. The speaker emphasizes that AI should be viewed as a tool rather than a creature, enhancing people's task performance. They acknowledge that some jobs may be automated, but they believe that new and more fulfilling jobs will emerge. Other participants in the video stress the importance of preparing the workforce for AI technologies and the need for transparency in understanding how these models function. The video also highlights the significance of scientists' involvement in AI system development and the need for transparency 

## 번역하기

pip install googletrans==3.1.0a0

In [18]:
from googletrans import Translator

def google_trans(messages):
    google = Translator()
    result = google.translate(messages, dest="ko")

    return result.text

result = google_trans(output)
print(result)

이 비디오에서는 초인적인 기계 지능이 일자리와 경제에 미치는 잠재적인 영향을 탐구합니다. 발표자는 기술 발전으로 인해 고용에 상당한 변화가 있을 것이라고 믿고 있지만, 정확한 결과를 예측하는 것은 어렵습니다. 그들은 이전의 기술 혁명으로 인해 새롭고 향상된 일자리가 창출되었다고 주장하며, 초인적인 기계 지능의 개발에서도 같은 일이 일어날 것으로 기대합니다. 발표자는 AI를 생물이 아닌 도구로 보아야 하며 사람들의 업무 수행 능력을 향상시켜야 한다고 강조합니다. 그들은 일부 직업이 자동화될 수 있다는 점을 인정하지만 새롭고 더 만족스러운 직업이 나타날 것이라고 믿습니다. 영상의 다른 참가자들은 AI 기술을 위한 인력 준비의 중요성과 이러한 모델의 작동 방식을 이해하는 데 있어 투명성의 필요성을 강조합니다. 이 영상은 또한 AI 시스템 개발에 과학자들이 참여하는 것의 중요성과 그들의 작업을 이해하는 데 있어 투명성의 필요성을 강조합니다. 과거 성과 이력이 미래 직업 전망을 보장하지 않으며 인공 일반 지능이 결국 인간 일자리의 상당 부분을 대체할 수 있지만 이를 달성하려면 아직 멀었다고 언급합니다. 연사는 인간이 더 나은 도구를 사용하여 새로운 기회를 발견할 것이라는 낙관론을 표현하지만 AI 기술이 잘못되면 심각한 해를 끼칠 가능성도 인정합니다.
